In [ ]:
import numpy as np
import random 
from collections import defaultdict
from CA import *

In [ ]:
def main_loop(P_init, 
              iterations=10, road_len=int(1e4), Vmax=6, Vrandom=True,
             p_gen=0.5):
    
    # make the initial road and cars
    cars, road = random_init_cars(road_len, P_init,
                                  Vrandom,
                                  Vmax)
    # make initial car positions
    positions = car_positions(road)
    
    # run the main experiment loop
    for i in range(1000):
        # update the position using current speed
        cars, road = position_update(positions, cars, road)
        # get the new car positions from the new array
        positions = car_positions(road)
        # update the speed based on the new position
        cars = speed_update(positions, cars, road, Vmax)
        print(road)
        
    return cars, road     
        
        
main_loop(1)        